In [1]:
"""
Sorting out dependencies

Given a list in this format:
---
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
---

Where each line is [name]: [value] and the value is either a number or an expression,
where the expression is a combination of the names of other lines, and the operators
are +, -, *, /.

The goal is to determine the value of the root line.
"""

import sys
import re



In [8]:
def parse_line(line):
    """
    Parse a line of the input file.
    Returns a tuple of (name, value, dependencies)
    """
    try:
        name, value = line.split(':')
        dependencies = []
        # if value is an int, we have no dependencies
        if not value.strip().isdigit():
            # find all the names in the value
            dependencies = re.findall('[a-z]{4}', value)
        return name.strip(), value.strip(), dependencies
    except Exception as e:
        print("Error parsing line: {}".format(line))
        raise e

def parse_yelling_monkeys(lines):
    """
    Parses a set of lines in to dependent monkeys.
    Returns a dictionary of {name: (value, dependencies)}
    """
    return {name: (value, dependencies) for name, value, dependencies in map(parse_line, lines)}

def parse_file(filename):
    """
    Parses a file of monkey yelling.
    Returns a dictionary of {name: (value, dependencies)}
    """
    with open(filename) as f:
        return parse_yelling_monkeys(f.readlines())

In [15]:
# monkeys is a dict keyed on the name of the monkey, with the value being a tuple of (value, dependencies)
# where value is either a number or an expression, and dependencies is a list of the names of the other monkeys referenced in the expression

# start from the 'root' monkey, and work our way down the tree, evaluating the expressions as we go
# we'll keep a dict of the values of the monkeys we've already evaluated, so we don't have to re-evaluate them
# we'll also keep a list of the monkeys we've evaluated, so we can detect circular dependencies

def evaluate_monkey(monkey, monkeys, monkey_values, evaluated_monkeys):
    """
    Evaluates the value of a monkey, and returns the value.
    """
    # if we've already evaluated this monkey, return the value
    if monkey in evaluated_monkeys:
        return monkey_values[monkey]
    # if the monkey has no dependencies, it's a number, so just return the value
    if not monkeys[monkey][1]:
        monkey_values[monkey] = int(monkeys[monkey][0])
        evaluated_monkeys.append(monkey)
        return monkey_values[monkey]
    # if we get here, we need to evaluate the expression
    # we'll do this by replacing the names of the dependencies with their values
    # and then evaluating the expression
    expression = monkeys[monkey][0]
    for dependency in monkeys[monkey][1]:
        expression = expression.replace(dependency, str(evaluate_monkey(dependency, monkeys, monkey_values, evaluated_monkeys)))
    # now we can evaluate the expression
    monkey_values[monkey] = eval(expression)
    evaluated_monkeys.append(monkey)
    return monkey_values[monkey]

monkeys = parse_file('./day21-input.txt')
evaluate_monkey('root', monkeys, {}, [])

41857219607906.0

In [32]:
"""
Rather than recursively evaluating the monkeys, let's first build a dependency graph to understand the structure of the monkeys.
"""

from pyvis.network import Network

def build_dependency_graph(monkeys):
    """
    Builds a dependency graph of the monkeys.
    Returns a pyvis.network.Network object
    """
    g = Network(notebook=True, cdn_resources='remote', height='100%', width='100%')
    for monkey in monkeys:
        color = 'gray'
        if monkey == 'root':
            color = 'green'
        elif monkey == 'humn':
            color = 'yellow'
        elif len(monkeys[monkey][1]) > 0:
            color = 'blue'
        g.add_node(monkey, label=monkey, color=color)
    for monkey in monkeys:
        for dependency in monkeys[monkey][1]:
            g.add_edge(monkey, dependency)
    return g

g = build_dependency_graph(monkeys)
g.barnes_hut(spring_length=0, spring_strength=0.115, damping=0.17)
g.show('day21-monkeyyelling.html')

In [62]:
"""
Given a monkey, we want to recursively build an expression that we can evaluate by expanding the dependencies in the formula. We'll keep a list of the monkeys we've already evaluated, so we can detect circular dependencies.

For example:

root: pppw + sjmn
pppw: cczh / lfqf
cczh: sllz + lgvd
sllz: 4
lfqf: 4
lgvd: ljgn * sllz
sjmn: sllz + lfqf

pppw = cczh / lfqf = (sllz + lgvd) / lfqf = (sllz + (ljgn * sllz)) / lfqf

"""

def build_expression(monkey, monkeys, evaluated_monkeys):
    """
    Recursively builds an expression for a monkey.
    """
    # if we've already evaluated this monkey, return the value
    if monkey in evaluated_monkeys:
        return monkey
    # if the monkey has no dependencies, it's a number, so just return the value
    if not monkeys[monkey][1]:
        evaluated_monkeys.append(monkey)
        return monkeys[monkey][0]
    # if we get here, we need to evaluate the expression
    # we'll do this by replacing the names of the dependencies with their values
    # and then evaluating the expression
    expression = monkeys[monkey][0]
    for dependency in monkeys[monkey][1]:
        expression = expression.replace(dependency, build_expression(dependency, monkeys, evaluated_monkeys))
    evaluated_monkeys.append(monkey)
    return '(' + expression + ')'

monkeys = parse_file('./day21-input.txt')

monkeys['root'] = (' = '.join(monkeys['root'][1]), monkeys['root'][1])
monkeys['humn'] = ('humn', [])

# build an equality equation for the root monkey
# eq_equation = build_expression('root', monkeys, [])
# print(eq_equation)

# from sympy import solve, Symbol, sympify

# # convert eq_equation to a sympy expression
# eq_expression = sympify(eq_equation)

expr1 = build_expression(monkeys['root'][1][0], monkeys, [])
expr2 = build_expression(monkeys['root'][1][1], monkeys, [])

print(expr1 + ' - ' + expr2)

from sympy import solve, Symbol, sympify

solve(expr1 + ' - ' + expr2, 'humn')

((((15 - 4) * 19) + (((2 * ((((((2 * (((14 + (3 + 6)) + 11) + (8 + 5))) / 2) * 3) - (19 + (9 + 10))) * 2) + (((4 + 4) * 2) + (5 * 11)))) + (((2 * 10) + (3 * 3)) * 10)) + ((((((2 * ((((((5 * 5) + 4) + 18) + ((2 * 5) + (9 * 3))) + 5) + (3 * 4))) * 3) * ((((((2 * 4) - 1) * 7) * 2) + (((2 + (((5 + 1) * 3) + 5)) * (8 * 2)) / 2)) * 3)) + ((((2 * ((((((2 * 17) + (7 * 2)) + ((6 * 4) + 12)) + (10 * (2 * 19))) + 3) * 2)) * 7) + ((((14 * 2) + (13 * 3)) - (2 * (5 + (2 * 3)))) * ((((((10 * 4) + ((3 * (8 + (2 * 7))) / 2)) * 2) / 2) * 7) - (2 * (((((8 - 1) + 1) * 4) + (6 * (2 * 3))) + (10 + 1)))))) * (9 + 10))) * (((((2 * ((((1 + (3 * 7)) + 1) * (4 * 2)) + (4 * 7))) / 8) * (9 - 1)) + (5 * (((((4 + (3 * 3)) * 2) / 2) * ((13 * 3) + 16)) + (((3 * 3) * 3) + (5 * (20 + 3)))))) * (((2 * ((2 * ((2 * ((7 * 5) + (((3 + 4) + 1) * (((((((4 * 3) * 4) - 1) + (2 * 16)) * 2) + (9 * (5 * 2))) / (2 * 4))))) / 2)) / 2)) + (11 * 7)) * 17))) - (2 * ((((2 * (2 * 3)) * 2) * (3 * (2 * 4))) + ((((((6 + 5) * (4 * (20 + 2))) 

[3916936880448]